# Introduction to Biopython, Jupyter, and Github

# Section 1: Biopython

[**Control flow**][cflow-wiki] refers to how the statements in a program *control* how the program *flows* (yeah....).  A basic example would be if you are looking for sequences shorter than 200nb in a FASTA file so that you can save them in another file.  In order to do this, you would:

* Read through each sequence in the file
* Determine if the sequence is shorter than 200nb.  If it is, write it to another file.

Each of these are considered *control statements*, the first being an example of a loop and the second a conditional.  We'll start with `for` loops.

[cflow-wiki]: https://en.wikipedia.org/wiki/Control_flow

## Section 1.1: Installing Biopython

### Exercise 1.1

## Section 1.2: Working With Sequence Objects


Sequence objects, or `Seq`s, are the foundation upon which Biopython is built.  A `Seq` object is made up of a `sequence` and an `alphabet`.

In [1]:
from Bio.Seq import Seq
from Bio.Alphabet import IUPAC
my_seq = Seq("GATCGATGGGCCTATATAGGATACGAAAATCGC", IUPAC.unambiguous_dna)
my_seq

Seq('GATCGATGGGCCTATATAGGATACGAAAATCGC', IUPACUnambiguousDNA())

In [2]:
print(my_seq)

GATCGATGGGCCTATATAGGATACGAAAATCGC


Because `Seq`s are subclasses of `str`, it inherits all of the `str` class methods.  For example:

In [3]:
len(my_seq)

33

In [4]:
my_seq[0]

'G'

In [5]:
my_seq[-1]

'C'

In [6]:
print(my_seq[0:20])

GATCGATGGGCCTATATAGG


In [7]:
for base in my_seq[:5]:
    print(base)

G
A
T
C
G


In [8]:
print(my_seq.lower())

gatcgatgggcctatataggatacgaaaatcgc


In [9]:
'ATA' in my_seq

True

We can also count the number of (non-overlapping) occurrences of a substring:

In [10]:
my_seq.count("ATA")

2

If you want to count the number of overlapping occurrences, use the `Seq.count_overlap` function.

In [11]:
my_seq.count_overlap("ATA")

3

We can use `str.count()` to calculate the GC content of a sequence:

In [12]:
(my_seq.count("G") + my_seq.count("C")) / len(my_seq)

0.45454545454545453

Calculating the GC content of a string in this way doesn't account for the ambiguous nucleotide S.  The `GC` function in `Bio.SeqUtils` can handle such cases.

In [13]:
ambig_string = Seq("ATGCRAGCTSGSTRSTGCGGCGASSGAGSARRRGSSA", IUPAC.ambiguous_dna)
(ambig_string.count("G") + ambig_string.count("C")) / len(ambig_string)

0.3783783783783784

In [14]:
from Bio.SeqUtils import GC
GC(ambig_string)

59.45945945945946

In [15]:
GC(my_seq)

45.45454545454545

You can also slice `Seq` objects as you would a string.  For instance, we can get the first ten characters:

In [16]:
ambig_string[:10]

Seq('ATGCRAGCTS', IUPACAmbiguousDNA())

We can print out every other character:

In [17]:
ambig_string[::2]

Seq('AGRGTGTSGGCASASRRSA', IUPACAmbiguousDNA())

Every third position:

In [18]:
ambig_string[::3]

Seq('ACGSTTGGSGRGA', IUPACAmbiguousDNA())

And we can shift our reading frame on the sequence:

In [19]:
ambig_string[1::3]

Seq('TRCGRGGAGSRS', IUPACAmbiguousDNA())

In [20]:
ambig_string[2::3]

Seq('GATSSCCSAARS', IUPACAmbiguousDNA())

We can also reverse the string:

In [21]:
ambig_string[::-1]

Seq('ASSGRRRASGAGSSAGCGGCGTSRTSGSTCGARCGTA', IUPACAmbiguousDNA())

Note that slicing a `Seq` returns another `Seq`, so anything you can do with a `Seq` you can do with a slice of a `Seq`.  

In [22]:
GC(ambig_string[:10])

50.0

There may be times where you just need a plain string version of a sequence; for instance, when writing to a file or a database.  

In [23]:
with open('test-1.1', 'w') as fout:
    fout.write(ambig_string)

TypeError: write() argument must be str, not Seq

In [24]:
with open('test-1.1', 'w') as fout:
    fout.write(str(ambig_string))

Note, though, that using string formatting avoids this problem since `format` coerces `Seq` objects to strings.

In [25]:
with open('test-1.1-2', 'w') as fout:
    fout.write("See? \n{}".format(ambig_string))

We can also concatenate sequences with `+` just as you would strings.  As you would expect, the result is also a sequence object.  We'll concatenate `my_seq`, which has the `IUPACUnambiguousDNA` alphabet, and `ambig_string`, which has the `IUPACAmbiguousDNA` alphabet.  Which alphabet will their concatenation have?  

In [26]:
my_seq + ambig_string

Seq('GATCGATGGGCCTATATAGGATACGAAAATCGCATGCRAGCTSGSTRSTGCGGC...SSA', IUPACAmbiguousDNA())

Since their concatenation has ambiguous bases, its alphabet is `IUPACAmbiguousDNA`.

Alphabets ensure that we only concatenate compatible sequences.  For example, let's create a protein sequence and try to concatenate it to `my_seq`:

In [27]:
my_prot = Seq("EVRNAK", IUPAC.protein)
my_prot

Seq('EVRNAK', IUPACProtein())

In [28]:
my_prot + my_seq

TypeError: Incompatible alphabets IUPACProtein() and IUPACUnambiguousDNA()

Because we told Biopython what kind of sequences `my_prot` and `my_seq` are by explicitly choosing their alphabet, it prevents us from combining the sequences.  

We can use a `for` loop to concatenate multiple sequences together:

In [29]:
sequences = [Seq("AGCGATGTTACGCATCAGGGCAGTCGCCCTAAAACAAAGTTAGGCCGC", IUPAC.unambiguous_dna),
            Seq("CCGGTTGGTAACGGCGCAGTGGCGGTTTTCAT", IUPAC.unambiguous_dna),
            Seq("CACAGCGGTTTTCAT", IUPAC.unambiguous_dna),
            Seq("AACGGCGCATGGCGGTTTTCAT", IUPAC.unambiguous_dna),
            Seq("AGTGGCGGTTTTCAT", IUPAC.unambiguous_dna)]

concatenated_sequences = ''
for seq in sequences:
    concatenated_sequences += seq
    
concatenated_sequences

Seq('AGCGATGTTACGCATCAGGGCAGTCGCCCTAAAACAAAGTTAGGCCGCCCGGTT...CAT', IUPACUnambiguousDNA())

#### Exercise 1.2.1

* Just a quick Python refresher - loop through `sequences` and print out "Sequence number ... is ... characters long."

`Seq`s have a lot of useful methods:

In [30]:
print([attr for attr in dir(my_seq) if not attr.startswith("_")])

['alphabet', 'back_transcribe', 'complement', 'count', 'count_overlap', 'endswith', 'find', 'lower', 'lstrip', 'reverse_complement', 'rfind', 'rsplit', 'rstrip', 'split', 'startswith', 'strip', 'tomutable', 'tostring', 'transcribe', 'translate', 'ungap', 'upper']


Because `Seq`s are subclasses of strings, they are immutable:

In [61]:
my_seq[0] = "T"

TypeError: 'Seq' object does not support item assignment

It can be useful sometimes to be able to change individual bases, though - say, if you're simulating mutations on a sequence.  The `tomutable` method provides a way of changing individual characters in a sequence:

In [57]:
my_mutable_seq = my_seq.tomutable()
my_mutable_seq

MutableSeq('GATCGATGGGCCTATATAGGATACGAAAATCGC', IUPACUnambiguousDNA())

Notice that the type changed from `Seq` to `MutableSeq`.  Now we can change the contents of the sequence:

In [63]:
my_mutable_seq[0] = 'T'
my_mutable_seq

MutableSeq('TATCGATGGGTCTATATAGGATACGAAAATCGC', IUPACUnambiguousDNA())

In [65]:
my_mutable_seq[-1] = "T"
my_mutable_seq

MutableSeq('TATCGATGGGTCTATATAGGATACGAAAATCGT', IUPACUnambiguousDNA())

And when you're done making changes to the string, you can convert it back to a `Seq` object via `.toseq`:

In [67]:
new_seq = my_mutable_seq.toseq()
new_seq

Seq('TATCGATGGGTCTATATAGGATACGAAAATCGT', IUPACUnambiguousDNA())

For example, we can transcribe and translate:

In [31]:
my_seq_transcript = my_seq.transcribe()
my_seq_transcript

Seq('GAUCGAUGGGCCUAUAUAGGAUACGAAAAUCGC', IUPACUnambiguousRNA())

Notice that the alphabet changed to `IUPACUnambiguousRNA`.  We can now translate this to protein: 

In [32]:
len(my_seq)

33

In [35]:
my_transcript_protein = my_seq_transcript.translate()
my_transcript_protein

Seq('DRWAYIGYENR', IUPACProtein())

We can also go straight from DNA to protein.  

In [36]:
coding_dna = Seq("ATGGCCATTGTAATGGGCCGCTGAAAGGGTGCCCGATAG", IUPAC.unambiguous_dna)
coding_dna.translate()

Seq('MAIVMGR*KGAR*', HasStopCodon(IUPACProtein(), '*'))

The `*` here indicates a stop codon - our sequence has an internal stop codon.  This isn't realistic, though - when a stop codon is encountered, translation is supposed to stop.  We can pass `True` to the `to_stop` argument of `translate()` to achieve this behavior:

In [37]:
coding_dna.translate(to_stop = True)

Seq('MAIVMGR', IUPACProtein())

We can be more realistic by telling Biopython to only start translation where it finds a start codon.  We do this by passing `cds = True` when we call `translate`.  Doing so tells Biopython that the sequence we are translating is a complete coding sequence (CDS), which means the sequence:

* has a whole number of codons (ie, the number of bases is a multiple of 3)
* begins with a start codon
* ends with a stop codon
* and has no internal stop codons

Let's see what happens when we try to pass `cds = True` when we translate `coding_dna`:

In [38]:
coding_dna.translate(cds = True)

TranslationError: Extra in frame stop codon found.

Since `coding_dna` has an internal stop codon, translation as a CDS fails.  Here's an example where it works:

In [40]:
cds_seq = Seq('ATGGCCATTGTAATGTAG', IUPAC.unambiguous_dna)
cds_seq.translate(cds = True)

Seq('MAIVM', IUPACProtein())

Remove `cds = True` and rerun the above code.  What's different in the output?

Now suppose we have a CDS that has a non-standard start codon.  For instance, `GTG` is a valid start codon for bacteria.  Take the following bacterial gene as an example:

In [44]:
from Bio.Alphabet import generic_dna
bac_gene = Seq("GTGAAAAAGATGCAATCTATCGTACTCGCACTTTCCCTGGTTCTGGTCGCTCCCATGGCA" + 
           "GCACAGGCTGCGGAAATTACGTTAGTCCCGTCAGTAAAATTACAGATAGGCGATCGTGAT" + 
           "AATCGTGGCTATTACTGGGATGGAGGTCACTGGCGCGACCACGGCTGGTGGAAACAACAT" + 
           "TATGAATGGCGAGGCAATCGCTGGCACCTACACGGACCGCCGCCACCGCCGCGCCACCAT" + 
           "AAGAAAGCTCCTCATGATCATCACGGCGGTCATGGTCCAGGCAAACATCACCGCTAA", generic_dna)

`bac_gene` is a valid CDS (check for yourself as an exercise); however, the table that `translate` uses for translation is the Standard Code ([NCBI table 1][ncbi-tables]), which doesn't recognize `GTG`as a start codon.  Thus, when we try to translate it using `cds = True`:

[ncbi-tables]: https://www.ncbi.nlm.nih.gov/Taxonomy/Utils/wprintgc.cgi

In [46]:
bac_gene.translate(cds = True)

TranslationError: First codon 'GTG' is not a start codon

However, we can tell `translate` to use a different table when performing translation via the `table` argument.  By default, this is set to `table = 1`.  Since we know our gene is a bacteria, we can tell `translate` to use the appropriate table.

#### Exercise 1.2.1
* Look through the translation tables on [NCBI][ncbi-tables] and determine which table we should use.  Translate `bac_gene` with the appropriate arguments.
* Alternatively,you can just call `translate` without providing a table or setting `cds = True`.  What's different?

[ncbi-tables]: https://www.ncbi.nlm.nih.gov/Taxonomy/Utils/wprintgc.cgi

### Exercise 1.2

## Section 1.3: SeqRecord Objects

Sequence Record objects, or `SeqRecord`s, can be thought of as database records about a sequence.  `SeqRecord`s are complex objects that have the following attributes:

* `.seq` - the `seq` of interest
* `.id` - a string identifying the `seq`
* `.name` - the "common" name for the `seq`
* `.description` - a human-readable description
* `.letter_annotations` - holds per-letter anotations about the letters in the sequence (eg, for quality scores).
* `.annotations` - a dictionary of additional information about the sequence
* `.features` - a list of `SeqFeature` objects containing more detailed information, (eg, positions of genes on a genmoe)
* `.dbxrefs` - a list of database cross-references as strings

You can access attribute `attr` directly. For example, if the name of your `SeqRecord` is `my_sr`, then you would type `my_sr.attr`.

You typically won't be creating `SeqRecord` objects by hand - we typically use `Bio.SeqIO` to read in a sequence file and create the records for you.  However, we'll do a couple examples as a demonstration:

In [ ]:
from Bio.SeqRecord import SeqRecord
my_seq_record = SeqRecord(my_seq)
my_seq_record

Since we didn't provide any additional information about the `my_seq_record`, all of the additional fields are blank.  However, we can assign values to them.

In [ ]:
my_seq_record.id = "WKM1213"
my_seq_record.description = "Just some dumb sequence I mashed out on the keyboard"
my_seq_record

You would normally provide these values when you create the `SeqRecord` object:

In [ ]:
my_seq_record = ''
my_seq_record = SeqRecord(my_seq, id = "WKM1213", description = "Did this work?")
my_seq_record

And here's an example of how you would write out a `SeqRecord` to a fasta file:

In [ ]:
with open('test-1.2', 'w') as fout:
    fout.write(">{} {}\n{}".format(
            my_seq_record.id, my_seq_record.description, my_seq_record.seq))

The `.letter_annotations` field contains a dictionary which lets you assign any list-like object that is as long as the sequence itself.  For example, we can save the [Phred quality scores][phred-wiki] for the sequence:

[phred-wiki]: https://en.wikipedia.org/wiki/Phred_quality_score

In [ ]:
import random
phred_scores = [random.randint(10, 60) for _ in range(len(my_seq))]
my_seq_record.letter_annotations['phred_quality'] = phred_scores
print(my_seq_record.letter_annotations['phred_quality'])

In [ ]:
for base_num, base in enumerate(my_seq_record.seq):
    print("{} {}".format(base, my_seq_record.letter_annotations['phred_quality'][base_num]))

`.letter_annotations` *only* takes list-like objects as long as the sequence itself.  If you attempt to assign an object that is not the same size, you get an error:

In [ ]:
my_seq_record.letter_annotations['Favorite foods'] = ['Spam', 'eggs', 'tofu', 'spinach', 'pineapple']

The `.annotation` attribute is a dictionary containing miscelleanous information.

In [ ]:
my_seq_record.annotations['Mood'] = "Big"
print(my_seq_record.annotations['Mood'])

### Exercise 1.3.1

* Find the average Phred quality score for `my_seq`.    

## Section 1.4: Working With Seq and SeqRecord Objects

### Exercise 1.4

## Section 1.5: Multiple Sequence Alignments

### Exercise 1.5